In [1]:
from bs4 import BeautifulSoup
import lxml
import requests
import pandas as pd
import numpy as np

In [2]:
def parse(full_url):
    page_content = BeautifulSoup(full_url.content, 'lxml')
    containers = page_content.findAll('article', class_="message")
    df = pd.DataFrame(columns = 
        ['message', 'user_name'])

    for item in containers:
        try:
            message = item.find('div', class_='bbWrapper').findAll(string=True, recursive=False)
        except:
            message = None

        try:
            user_name = item.find('a', class_='username').text
        except:
            rating_title = None

        df = pd.concat(
            [df, pd.DataFrame(
                [{'message': ' '.join(message), 
                'user_name': user_name
                }]
            )], ignore_index=True
        )
    return df

In [5]:
headers = {'User-Agent': ''}
base_url = 'https://thefarmingforum.co.uk/index.php?threads/milk-price-tracker.53178/page-'
all_messages_df = pd.DataFrame(columns = 
    ['message', 'user_name'])

page_num=1
# you can adjust this number on how many reviews you which to scrape
while True and page_num<100:
    
    full_url = base_url+ str(page_num)
    get_url = requests.get(full_url, headers=headers, timeout=10)
    if(get_url.history and page_num>1):
        break

    partial_messages_df = parse(get_url)
    all_messages_df = pd.concat(
        [all_messages_df, partial_messages_df], ignore_index=True
    )
    page_num += 1

In [6]:
all_messages_df

,message,user_name
0,Why no milk price tracker thread? \nWe have th...,rc mx110
1,Christ no!!!! \nIts bad enough knowing what im...,buffalo_soldier
2,"I would be willing to, be interesting to see v...",rc mx110
3,Looks like you just started it.,Robigus
4,\nGrr . Lower,buffalo_soldier
...,...,...
1975,\n5 years of 14 to 17 is unsustainable (unles...,Only
1976,"I don't think the talk we had recently , of th...",Cowmangav
1977,We are slow at responding though too many thou...,Only
1978,Oh great.Now Brussels will allow state aid fo...,Cowmangav


In [7]:
all_messages_df.to_pickle('messages_scraped.pkl')